In [1]:
# Installing the needed dependencies
import sys
#!{sys.executable} -m pip install imblearn

In [2]:
# Importing the Needed Dependencies
from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.metrics import balanced_accuracy_score, confusion_matrix, classification_report
from sklearn.utils.validation import check_array
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from pprint import pprint
import csv
from imblearn.over_sampling import RandomOverSampler

## Loading "ETL_credit_data.csv" Into Dataframe

In [3]:
# Create reference to CSV file
csv_path = Path("../output_data/ETL/csv/ETL_credit_data.csv")

# Import the CSV into a pandas DataFrame
credit_data_df = pd.read_csv(csv_path)

# Display dataframe
credit_data_df.head()

,Current Loan Amount,Credit Score,Annual Income,Monthly Income,Monthly Debt,Debt / Income Ratio,Credit History(Years),Months Delinquent,Open Accounts,Credit Problems,Credit Balance,Total Credit,Credit Usage Ratio,Bankruptcies,Loan Purpose
0,611314,747.0,2074116.0,172843.00,42000.83,0.243000,21.8,0.0,9,0,621908,1058970,0.587276,0.0,Debt Consolidation
1,266662,734.0,1919190.0,159932.50,36624.40,0.228999,19.4,0.0,11,0,679573,904442,0.751373,0.0,Debt Consolidation
2,153494,709.0,871112.0,72592.67,8391.73,0.115600,12.5,10.0,10,0,38532,388036,0.099300,0.0,Debt Consolidation
3,176242,727.0,780083.0,65006.92,16771.87,0.258001,16.5,27.0,16,1,156940,531322,0.295376,1.0,Debt Consolidation
4,321992,744.0,1761148.0,146762.33,39478.77,0.268998,26.0,44.0,14,0,359765,468072,0.768610,0.0,Debt Consolidation


## Split the Data into Training and Testing Sets

### Step 1: Create the labels set (`y`)  from the “Bankruptcies” column, and then create the features (`X`) DataFrame from the remaining columns

In [4]:
# Separate the data into labels and features
# Separate the y variable, the labels
y = credit_data_df['Bankruptcies']

# Separate the X variable, the features
X = credit_data_df.drop(columns='Bankruptcies')

In [5]:
# Review the y variable Series
y.head()

0    0.0
1    0.0
2    0.0
3    1.0
4    0.0
Name: Bankruptcies, dtype: float64

In [6]:
# Review the X variable DataFrame
X.head()

,Current Loan Amount,Credit Score,Annual Income,Monthly Income,Monthly Debt,Debt / Income Ratio,Credit History(Years),Months Delinquent,Open Accounts,Credit Problems,Credit Balance,Total Credit,Credit Usage Ratio,Loan Purpose
0,611314,747.0,2074116.0,172843.00,42000.83,0.243000,21.8,0.0,9,0,621908,1058970,0.587276,Debt Consolidation
1,266662,734.0,1919190.0,159932.50,36624.40,0.228999,19.4,0.0,11,0,679573,904442,0.751373,Debt Consolidation
2,153494,709.0,871112.0,72592.67,8391.73,0.115600,12.5,10.0,10,0,38532,388036,0.099300,Debt Consolidation
3,176242,727.0,780083.0,65006.92,16771.87,0.258001,16.5,27.0,16,1,156940,531322,0.295376,Debt Consolidation
4,321992,744.0,1761148.0,146762.33,39478.77,0.268998,26.0,44.0,14,0,359765,468072,0.768610,Debt Consolidation


### Step 2: Check the balance of the labels variable (`y`) by using the `value_counts` function

In [7]:
# Check the balance of our target values
y.value_counts()

Bankruptcies
0.0    8917
1.0    1083
Name: count, dtype: int64

### Step 3: Split the data into training and testing datasets by using `train_test_split`

In [8]:
# Import the train_test_learn module
from sklearn.model_selection import train_test_split

# Split the data using train_test_split
# Assign a random_state of 1 to the function
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1)

## Create a Logistic Regression Model with the Original Data

###  Step 1: Fit a logistic regression model by using the training data (`X_train` and `y_train`)

In [9]:
# Import the LogisticRegression module from SKLearn
from sklearn.linear_model import LogisticRegression

# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
Log_Reg_Model = LogisticRegression(random_state = 1)

# Fit the model using training data
Log_Reg_Model.fit(X_train, y_train)

ValueError: could not convert string to float: 'Other'